In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, LongType, DecimalType

spark # Printing Spark Session's details

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
# Read data from Snowflake table into dataframe

listings_df = (spark.read
  .format("snowflake")
  .option("sfURL", "****************************")
  .option("user", "*********************")
  .option("password", "****************")
  .option("database", "AirbnbData")
  .option("schema", "listings")
  .option("dbtable", "listings")
  .load()
)

In [ ]:
listings_df.printSchema() # Schema check

root
 |-- ID: decimal(38,0) (nullable = true)
 |-- NAME: string (nullable = true)
 |-- HOST_ID: decimal(38,0) (nullable = true)
 |-- HOST_NAME: string (nullable = true)
 |-- NEIGHBOURHOOD_GROUP: double (nullable = true)
 |-- NEIGHBOURHOOD: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ROOM_TYPE: string (nullable = true)
 |-- PRICE: decimal(38,0) (nullable = true)
 |-- MINIMUM_NIGHTS: decimal(38,0) (nullable = true)
 |-- NUMBER_OF_REVIEWS: decimal(38,0) (nullable = true)
 |-- LAST_REVIEW: string (nullable = true)
 |-- REVIEWS_PER_MONTH: double (nullable = true)
 |-- CALCULATED_HOST_LISTINGS_COUNT: decimal(38,0) (nullable = true)
 |-- AVAILABILITY_365: decimal(38,0) (nullable = true)
 |-- NUMBER_OF_REVIEWS_LTM: decimal(38,0) (nullable = true)
 |-- LICENSE: double (nullable = true)



In [ ]:
display(listings_df.limit(10)) # Displaying top 10 rows

ID,NAME,HOST_ID,HOST_NAME,NEIGHBOURHOOD_GROUP,NEIGHBOURHOOD,LATITUDE,LONGITUDE,ROOM_TYPE,PRICE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS,LAST_REVIEW,REVIEWS_PER_MONTH,CALCULATED_HOST_LISTINGS_COUNT,AVAILABILITY_365,NUMBER_OF_REVIEWS_LTM,LICENSE
106332,"Lovely large room, Bethnal Green",264345,Susie,null,Tower Hamlets,51.52874,-0.05396,Private room,55,3,65,2016-06-10,0.47,1,74,0,null
13913,Holiday London DB Room Let-on going,54730,Alina,null,Islington,51.56861,-0.1127,Private room,50,1,30,2022-07-15,0.2,2,343,9,null
284532,COSY STUDIO-FLAT WITH A GREAT VIEW,1481412,Luca,null,Tower Hamlets,51.52851,-0.07426,Entire home/apt,90,5,42,2022-01-04,0.34,1,222,1,null
107048,A Luxury Studio Suite in Clerkenwell,259088,Simon,null,Islington,51.52478,-0.10484,Private room,180,4,493,2022-09-02,3.59,5,236,20,null
284603,Notting Hill church stay in Heaven!,1481851,Tania,null,Kensington and Chelsea,51.51464,-0.20004,Entire home/apt,297,14,6,2022-07-04,0.05,1,180,2,null
15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,null,Kensington and Chelsea,51.4878,-0.16813,Entire home/apt,75,3,89,2020-03-16,0.57,1,70,0,null
107051,JESSIE the narrowboat in Little Venice,554519,Meg,null,Westminster,51.52055,-0.18317,Entire home/apt,204,2,581,2022-09-02,4.26,1,193,66,null
17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,null,Westminster,51.52195,-0.14094,Entire home/apt,379,4,51,2022-09-06,0.36,5,249,8,null
286675,APARTMENT IN NORTH WEST LONDON,1491611,Nora,null,Harrow,51.57982,-0.37539,Entire home/apt,90,3,104,2022-08-22,0.86,1,318,4,null
116268,DOUBLE ROOM NEAR CENTRAL LONDON,586671,Joe,null,Enfield,51.61137,-0.11888,Private room,30,7,32,2022-09-08,0.29,1,251,4,null


In [ ]:
# Dropping 2 columns as they don't have any values - excessive missing values

columns_to_drop = ['neighbourhood_group', 'license'] # Specify column names to drop
listings_df = listings_df.drop(*columns_to_drop)
display(listings_df.limit(2)) # Display data to check

ID,NAME,HOST_ID,HOST_NAME,NEIGHBOURHOOD,LATITUDE,LONGITUDE,ROOM_TYPE,PRICE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS,LAST_REVIEW,REVIEWS_PER_MONTH,CALCULATED_HOST_LISTINGS_COUNT,AVAILABILITY_365,NUMBER_OF_REVIEWS_LTM
106332,"Lovely large room, Bethnal Green",264345,Susie,Tower Hamlets,51.52874,-0.05396,Private room,55,3,65,2016-06-10,0.47,1,74,0
13913,Holiday London DB Room Let-on going,54730,Alina,Islington,51.56861,-0.1127,Private room,50,1,30,2022-07-15,0.2,2,343,9


In [ ]:
listings_df = listings_df.dropDuplicates(['id']) # Removing duplicates based on 'id' column

In [ ]:
# Check null count in each column

null_count = listings_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in listings_df.columns
])

null_count.display()

ID,NAME,HOST_ID,HOST_NAME,NEIGHBOURHOOD,LATITUDE,LONGITUDE,ROOM_TYPE,PRICE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS,LAST_REVIEW,REVIEWS_PER_MONTH,CALCULATED_HOST_LISTINGS_COUNT,AVAILABILITY_365,NUMBER_OF_REVIEWS_LTM
0,21,0,5,0,0,0,0,0,0,0,16780,16780,0,0,0


In [ ]:
# Instead of filling, we are dropping rows having null values

listings_df = listings_df.dropna(subset=['NAME', 'HOST_NAME', 'LAST_REVIEW', 'REVIEWS_PER_MONTH']) # Specify columns that have null values

In [ ]:
# Rechecking null values

null_count = listings_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in listings_df.columns
])

null_count.display()

ID,NAME,HOST_ID,HOST_NAME,NEIGHBOURHOOD,LATITUDE,LONGITUDE,ROOM_TYPE,PRICE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS,LAST_REVIEW,REVIEWS_PER_MONTH,CALCULATED_HOST_LISTINGS_COUNT,AVAILABILITY_365,NUMBER_OF_REVIEWS_LTM
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Since data cleansing is done, we can change to respective dtypes

listings_df = listings_df.withColumn("id", listings_df["id"].cast(StringType())) \
                         .withColumn("name", listings_df["name"].cast(StringType())) \
                         .withColumn("host_id", listings_df["host_id"].cast(IntegerType())) \
                         .withColumn("host_name", listings_df["host_name"].cast(StringType())) \
                         .withColumn("neighbourhood", listings_df["neighbourhood"].cast(StringType())) \
                         .withColumn("latitude", listings_df["latitude"].cast(FloatType())) \
                         .withColumn("longitude", listings_df["longitude"].cast(FloatType())) \
                         .withColumn("room_type", listings_df["room_type"].cast(StringType())) \
                         .withColumn("price", listings_df["price"].cast(FloatType())) \
                         .withColumn("minimum_nights", listings_df["minimum_nights"].cast(IntegerType())) \
                         .withColumn("number_of_reviews", listings_df["number_of_reviews"].cast(IntegerType())) \
                         .withColumn("last_review", listings_df["last_review"].cast(DateType())) \
                         .withColumn("reviews_per_month", listings_df["reviews_per_month"].cast(FloatType())) \
                         .withColumn("calculated_host_listings_count", listings_df["calculated_host_listings_count"].cast(IntegerType())) \
                         .withColumn("availability_365", listings_df["availability_365"].cast(IntegerType())) \
                         .withColumn("number_of_reviews_ltm", listings_df["number_of_reviews_ltm"].cast(IntegerType()))

In [ ]:
listings_df.printSchema() # Rechecking our schema

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews_ltm: integer (nullable = true)



In [ ]:
# Creating a column 'price_range_bin' that bins the listing's prices into ranges
# Greater than 5000 is 'Very High', greater than 1000 & lesser than 5000 is 'High
# Inbetween 500 and 1000 is 'Medium' & inbetween 0 and 500 is 'Low'

listings_df = listings_df.withColumn("price_range_bin", when(col("price") > 5000, "Very High") \
                         .when((col("price") >= 1000) & (col("price") <= 5000), "High") \
                         .when((col("price") >= 500) & (col("price") <= 1000), "Medium") \
                         .when((col("price") > 1) & (col("price") <= 500), "Low") \
                         .otherwise("Out of Range"))

display(listings_df.limit(5))

id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,price_range_bin
13913,Holiday London DB Room Let-on going,54730,Alina,Islington,51.56861,-0.1127,Private room,50.0,1,30,2022-07-15,0.2,2,343,9,Low
47192,Single Room in zone 1,212734,"Let'S Dance, Put On Your Red Shoes",Camden,51.52958,-0.14344,Private room,48.0,2,419,2022-09-04,2.88,1,318,44,Low
49970,Beautiful Small Studio Hammersmith,216660,Boris,Hammersmith and Fulham,51.49462,-0.22923,Hotel room,72.0,1,110,2022-08-07,0.77,11,229,20,Low
70482,Fabulous Hackney Flat Victoria Park London fields,357352,Michael,Hackney,51.54096,-0.0487,Entire home/apt,190.0,5,2,2022-08-31,0.78,1,362,2,Low
78892,Bright single room in family home.,424230,Julia,Southwark,51.46478,-0.08007,Private room,40.0,7,58,2022-05-15,0.64,1,2,2,Low


In [ ]:
# Creating column 'days_since_last_review' calculating days since last review

listings_df = listings_df \
        .withColumn("current_date", current_date()) \
        .withColumn("days_since_last_review", \
        datediff(col("current_date"), \
        col("last_review"))) \
        .drop("current_date") # Drops 'current_date' column after calculation

display(listings_df.limit(5))

id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,price_range_bin,days_since_last_review
13913,Holiday London DB Room Let-on going,54730,Alina,Islington,51.56861,-0.1127,Private room,50.0,1,30,2022-07-15,0.2,2,343,9,Low,753
47192,Single Room in zone 1,212734,"Let'S Dance, Put On Your Red Shoes",Camden,51.52958,-0.14344,Private room,48.0,2,419,2022-09-04,2.88,1,318,44,Low,702
49970,Beautiful Small Studio Hammersmith,216660,Boris,Hammersmith and Fulham,51.49462,-0.22923,Hotel room,72.0,1,110,2022-08-07,0.77,11,229,20,Low,730
70482,Fabulous Hackney Flat Victoria Park London fields,357352,Michael,Hackney,51.54096,-0.0487,Entire home/apt,190.0,5,2,2022-08-31,0.78,1,362,2,Low,706
78892,Bright single room in family home.,424230,Julia,Southwark,51.46478,-0.08007,Private room,40.0,7,58,2022-05-15,0.64,1,2,2,Low,814


In [ ]:
# Creating a column 'week_of_year' and check the year's week number

listings_df = listings_df.withColumn("week_of_year", weekofyear(listings_df.last_review))
display(listings_df.limit(5))

id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,price_range_bin,days_since_last_review,week_of_year
13913,Holiday London DB Room Let-on going,54730,Alina,Islington,51.56861,-0.1127,Private room,50.0,1,30,2022-07-15,0.2,2,343,9,Low,753,28
47192,Single Room in zone 1,212734,"Let'S Dance, Put On Your Red Shoes",Camden,51.52958,-0.14344,Private room,48.0,2,419,2022-09-04,2.88,1,318,44,Low,702,35
49970,Beautiful Small Studio Hammersmith,216660,Boris,Hammersmith and Fulham,51.49462,-0.22923,Hotel room,72.0,1,110,2022-08-07,0.77,11,229,20,Low,730,31
70482,Fabulous Hackney Flat Victoria Park London fields,357352,Michael,Hackney,51.54096,-0.0487,Entire home/apt,190.0,5,2,2022-08-31,0.78,1,362,2,Low,706,35
78892,Bright single room in family home.,424230,Julia,Southwark,51.46478,-0.08007,Private room,40.0,7,58,2022-05-15,0.64,1,2,2,Low,814,19


In [ ]:
# Creates column 'review_to_availability_ratio'
# Checks how active a listing is in terms of reviews compared to its availability

listings_df = listings_df.withColumn("review_to_availability_ratio", 
                         when(col("availability_365") > 0, col("number_of_reviews") / col("availability_365"))
                         .otherwise(lit(0)))

display(listings_df.limit(5))

id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,price_range_bin,days_since_last_review,week_of_year,review_to_availability_ratio
13913,Holiday London DB Room Let-on going,54730,Alina,Islington,51.56861,-0.1127,Private room,50.0,1,30,2022-07-15,0.2,2,343,9,Low,753,28,0.08746355685131195
47192,Single Room in zone 1,212734,"Let'S Dance, Put On Your Red Shoes",Camden,51.52958,-0.14344,Private room,48.0,2,419,2022-09-04,2.88,1,318,44,Low,702,35,1.3176100628930818
49970,Beautiful Small Studio Hammersmith,216660,Boris,Hammersmith and Fulham,51.49462,-0.22923,Hotel room,72.0,1,110,2022-08-07,0.77,11,229,20,Low,730,31,0.48034934497816595
70482,Fabulous Hackney Flat Victoria Park London fields,357352,Michael,Hackney,51.54096,-0.0487,Entire home/apt,190.0,5,2,2022-08-31,0.78,1,362,2,Low,706,35,0.0055248618784530384
78892,Bright single room in family home.,424230,Julia,Southwark,51.46478,-0.08007,Private room,40.0,7,58,2022-05-15,0.64,1,2,2,Low,814,19,29.0


In [ ]:
# Check average price in each neighbourhood

avg_price_per_neighbourhood = listings_df \
            .groupby("neighbourhood") \
            .agg({"price": "avg"}) \
            .withColumnRenamed("avg(price)", "avg_price")

display(avg_price_per_neighbourhood.limit(10))

neighbourhood,avg_price
Wandsworth,152.4884775217227
Croydon,74.59482758620689
Bexley,81.65423728813559
Lambeth,125.04055896387185
Barking and Dagenham,90.17105263157895
Camden,183.12678331090174
Greenwich,110.46307558644656
Newham,150.960231980116
Tower Hamlets,122.1356251471627
Barnet,112.18778486782134


In [ ]:
# Check average price for each type of room

avg_price_roomtype = listings_df \
            .groupby("room_type") \
            .agg({"price": "avg"}) \
            .withColumnRenamed("avg(price)", "avg_price")

display(avg_price_roomtype)

room_type,avg_price
Shared room,57.58921161825726
Hotel room,241.03333333333333
Entire home/apt,205.38230593249554
Private room,80.87393585428627


In [ ]:
# What are the top 10 most expensive listing

top_10_expensive_listing = listings_df \
            .orderBy(desc("price")) \
            .limit(10)

display(top_10_expensive_listing)

id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,price_range_bin,days_since_last_review,week_of_year,review_to_availability_ratio
33472290,"Budget single room near Stratford, East London",223188053,Bikash,Newham,51.55034,0.00932,Private room,23000.0,2,28,2022-08-24,2.33,2,18,28,Very High,713,34,1.5555555555555556
22877483,Stratford Saffron Apartment,148118580,Baronial Pads,Newham,51.54624,0.00689,Entire home/apt,20362.0,1,8,2022-03-20,0.17,1,90,3,Very High,870,11,0.08888888888888889
35511672,"2 bed flat close to Stamford Hill,Stoke Newington",77446666,Karolina,Hackney,51.56686,-0.06476,Entire home/apt,10000.0,100,16,2020-02-09,0.41,5,0,0,Very High,1640,6,0.0
19370148,Modern 2BR Flat with Garden by Clapham Common/Battersea,135783855,Alison,Wandsworth,51.46275,-0.16304,Entire home/apt,10000.0,3,40,2020-03-17,0.67,1,365,0,Very High,1603,12,0.1095890410958904
44696103,Cheap Single Rooms in Carlton Hotel,89693959,Azimali,Westminster,51.49101,-0.13753,Private room,8591.0,1,43,2022-09-06,1.78,6,355,27,Very High,700,36,0.12112676056338029
9470827,"Room in a cosy flat. Central, clean",5484135,David,Camden,51.55419,-0.1438,Private room,8000.0,2,8,2016-09-10,0.1,1,365,0,Very High,2887,36,0.021917808219178082
22965536,Period Victorian house with garden,45230234,Matthew,Haringey,51.57254,-0.10537,Private room,8000.0,1124,7,2019-08-28,0.12,1,0,0,Very High,1805,35,0.0
33864869,Chambre Double maison vegetarienne,43019905,Remy,Haringey,51.576,-0.10944,Private room,7852.0,1,1,2019-05-27,0.02,1,365,0,Very High,1898,22,0.0027397260273972603
36657089,Westminster Central Victorian Cosy Apartment,194367701,Daniel,Westminster,51.48606,-0.13853,Entire home/apt,7800.0,4,7,2019-09-07,0.18,4,0,0,Very High,1795,36,0.0
17713105,Huge Bright room Spacious Hampstead Penthouse Flat,11511561,Balbinder,Camden,51.54718,-0.1761,Private room,7551.0,90,1,2017-05-01,0.02,2,365,0,Very High,2654,18,0.0027397260273972603


In [ ]:
# Top 5 hosts who have the highest average price for their listings and the highest number of listings they manage

host_agg_df = listings_df.groupBy('host_id', 'host_name') \
    .agg(
        avg('price').alias('average_price'),
        count('id').alias('number_of_listings')
    )

# Define window specification to rank hosts by average price in descending order
window = Window.orderBy(host_agg_df['average_price'].desc())

# We'll use row number to rank hosts
host_ranked_df = host_agg_df.withColumn('rank', row_number().over(window))

# Get top 5 hosts
top_5_hosts_df = host_ranked_df.filter(host_ranked_df['rank'] <= 5)

display(top_5_hosts_df)

host_id,host_name,average_price,number_of_listings,rank
223188053,Bikash,23000.0,1,1
148118580,Baronial Pads,20362.0,1,2
135783855,Alison,10000.0,1,3
45230234,Matthew,8000.0,1,4
5484135,David,8000.0,1,5


In [ ]:
# Check average price of host and room type, correlates with number of listings each host has for each room type

# Calculate average price by host and room type
price_summary_df = listings_df.groupBy('host_id', 'host_name', 'room_type') \
    .agg(
        avg('price').alias('average_price')
    )

# Count number of listings by host and room type
listing_count_df = listings_df.groupBy('host_id', 'host_name', 'room_type') \
    .agg(
        count('id').alias('number_of_listings')
    )

# Join and do correlation
joined_df = price_summary_df.join(listing_count_df, on=['host_id', 'host_name', 'room_type'])

# # Compute correlation between average price and number of listings for each room type
correlation_df = joined_df.groupBy('room_type').agg(
    corr('average_price', 'number_of_listings').alias('price_listings_correlation')
)

display(correlation_df)

room_type,price_listings_correlation
Shared room,-0.08314025616562573
Hotel room,0.1941185129896555
Entire home/apt,0.05923510613958202
Private room,0.038548986123491155


In [ ]:
# Write transformed dataframe back to Snowflake into different table

options = {
    "sfURL": "**********************************************",
    "sfUser": "****************",
    "sfPassword": "****************",
    "sfDatabase": "AirbnbData",
    "sfSchema": "listings",
    "sfRole": "ACCOUNTADMIN"
}

try:
    listings_df.write.format("snowflake") \
        .options(**options) \
        .option("dbtable", "airbnb_listings_transformed") \
        .mode("overwrite") \
        .save()
    
    print("Data loaded to Snowflake successfully")
except Exception as e:
    print(f"Error loading data to Snowflake: {e}")

Data loaded to Snowflake successfully
